In [1]:
!pip install transformers torch torchtext

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.6/7.6 MB 37.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 294.9/294.9 kB 25.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.8/7.8 MB 38.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.3/1.3 MB 56.1 MB/s eta 0:00:00


In [1]:
train_data_path = "/content/srWaC_transformed_train.txt"
test_data_path = "/content/srWaC_transformed_test.txt"

def load_data(data_path):
    with open(data_path, "r", encoding="utf-8") as file:
        lines = file.readlines()
    sentences = [line.strip().split() for line in lines]
    return sentences

train_sentences = load_data(train_data_path)
test_sentences = load_data(test_data_path)

In [3]:
from transformers import BertTokenizer

# Koristićemo BERT model posebno treniran za srpski jezik
tokenizer = BertTokenizer.from_pretrained("bert-base-uncased")

def tokenize_sentences(sentences):
    tokenized = []
    for sentence in sentences:
        tokens = []
        labels = []
        for pair in sentence:
            parts = pair.split("/")
            if len(parts) == 2:  # Očekujemo tačno dva dela
                word, label = parts
                tokens.extend(tokenizer.tokenize(word))
                labels.extend([label] + ["Prazno"] * (len(tokenizer.tokenize(word)) - 1))
            else:
                # Ovde možete rukovati redovima koji ne ispunjavaju očekivanu strukturu
                print("Greška: Red ne ispunjava očekivanu strukturu:", pair)
        tokenized.append((tokens, labels))
    return tokenized

train_tokenized = tokenize_sentences(train_sentences)
test_tokenized = tokenize_sentences(test_sentences)


Greška: Red ne ispunjava očekivanu strukturu: http://www.water-energy-food.org//X
Greška: Red ne ispunjava očekivanu strukturu: //Z
Greška: Red ne ispunjava očekivanu strukturu: //Z
Greška: Red ne ispunjava očekivanu strukturu: http://www.ebay.com/itm/Western-Digital-My-Passport-Edge-Mac-500GB-/310650415709?_trksid=p2050601.m2372&amp;_trkparms=aid%3D111000%26algo%3DREC.CURRENT%26ao%3/X


In [4]:
from torch.utils.data import DataLoader, TensorDataset
import torch

labels_srWaC = ['AGA', 'AGD', 'AGG', 'AGI', 'AGL', 'AGN', 'AGV',
	'APA', 'APD', 'APG', 'API', 'APL', 'APN', 'APV',
	'ASA', 'ASD', 'ASG', 'ASI', 'ASL', 'ASN',
	'CC', 'CS',
	'I',
	'MC', 'MCA', 'MCD', 'MCG', 'MCI', 'MCL', 'MCN',
	'MM',
	'MO', 'MOA', 'MOD', 'MOG', 'MOI', 'MOL', 'MON',
	'MS', 'MSI', 'MSL', 'MSN',
	'NA', 'ND', 'NG', 'NI', 'NL', 'NN', 'NV',
	'PNA', 'PND', 'PNG', 'PNI', 'PNL', 'PNN',
	'PPA', 'PPD', 'PPG', 'PPI', 'PPL', 'PPN', 'PPV',
	'PXA', 'PXD', 'PXG', 'PXI', 'PXL',
	'QO', 'QQ', 'QR', 'QZ',
	'RG', 'RR', 'RS',
	'SA', 'SD', 'SG', 'SI', 'SL',
	'VAA', 'VAE', 'VAM', 'VAN', 'VAP', 'VAR',
	'VMA', 'VME', 'VMF', 'VMM', 'VMN', 'VMP', 'VMR',
	'X', 'XF', 'Y', 'Z', 'Prazno']

max_token_length = max(len(tokens) for tokens, _ in train_tokenized + test_tokenized)

print(max_token_length)


400


In [5]:
def prepare_data(tokenized_data, tokenizer, max_length):
    input_ids = []
    attention_masks = []
    labels = []

    for tokens, label_ids in tokenized_data:
        # Provera da li su tokens i label_ids prazni
        if not tokens or not label_ids:
            continue

        # Mapiranje nepostojećih oznaka na "Prazno"
        label_ids = [label if label in labels_srWaC else "Prazno" for label in label_ids]

        encoded = tokenizer.encode_plus(tokens, padding="max_length", max_length=max_length, truncation=True, add_special_tokens=True)
        input_ids.append(encoded["input_ids"])
        attention_masks.append(encoded["attention_mask"])

        # Dodajte "Prazno" oznake na kraju liste label_ids kako biste dobili uniformnu dužinu
        while len(label_ids) < max_length:
            label_ids.append("Prazno")

        label_ids = [labels_srWaC.index(label) for label in label_ids]
        labels.append(label_ids)

    input_ids = torch.tensor(input_ids)
    attention_masks = torch.tensor(attention_masks)
    labels = torch.tensor(labels)

    dataset = TensorDataset(input_ids, attention_masks, labels)
    return dataset


# Prvo pronađite maksimalnu dužinu tokena
max_token_length = max(len(tokens) for tokens, _ in train_tokenized + test_tokenized)

# Zatim koristite tu maksimalnu dužinu za tokenizaciju i kodiranje
train_dataset = prepare_data(train_tokenized, tokenizer, max_length=max_token_length)
test_dataset = prepare_data(test_tokenized, tokenizer, max_length=max_token_length)


batch_size = 8
train_dataloader = DataLoader(train_dataset, batch_size=batch_size)
test_dataloader = DataLoader(test_dataset, batch_size=batch_size)

In [6]:
from transformers import BertForTokenClassification, AdamW

model = BertForTokenClassification.from_pretrained("bert-base-uncased", num_labels=len(labels_srWaC))

optimizer = AdamW(model.parameters(), lr=1e-5)

# Postavite broj epoha i druge hiperparametre prema potrebi
num_epochs = 5

for epoch in range(num_epochs):
    model.train()
    for batch in train_dataloader:
        input_ids, attention_mask, labels = batch
        outputs = model(input_ids, attention_mask=attention_mask, labels=labels)
        loss = outputs.loss
        loss.backward()
        optimizer.step()
        optimizer.zero_grad()

# Evaluacija modela
model.eval()
# Dodajte kod za evaluaciju na test skupu podataka

Some weights of BertForTokenClassification were not initialized from the model checkpoint at bert-base-uncased and are newly initialized: ['classifier.bias', 'classifier.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.
/usr/local/lib/python3.10/dist-packages/transformers/optimization.py:411: FutureWarning: This implementation of AdamW is deprecated and will be removed in a future version. Use the PyTorch implementation torch.optim.AdamW instead, or set `no_deprecation_warning=True` to disable this warning
  warnings.warn(


BertForTokenClassification(
  (bert): BertModel(
    (embeddings): BertEmbeddings(
      (word_embeddings): Embedding(30522, 768, padding_idx=0)
      (position_embeddings): Embedding(512, 768)
      (token_type_embeddings): Embedding(2, 768)
      (LayerNorm): LayerNorm((768,), eps=1e-12, elementwise_affine=True)
      (dropout): Dropout(p=0.1, inplace=False)
    )
    (encoder): BertEncoder(
      (layer): ModuleList(
        (0-11): 12 x BertLayer(
          (attention): BertAttention(
            (self): BertSelfAttention(
              (query): Linear(in_features=768, out_features=768, bias=True)
              (key): Linear(in_features=768, out_features=768, bias=True)
              (value): Linear(in_features=768, out_features=768, bias=True)
              (dropout): Dropout(p=0.1, inplace=False)
            )
            (output): BertSelfOutput(
              (dense): Linear(in_features=768, out_features=768, bias=True)
              (LayerNorm): LayerNorm((768,), eps=1e-12, el

In [14]:
import os

# Putanja do direktorijuma u kojem želite da sačuvate model
save_dir = r"/content/"

# Kreirajte direktorijum ako ne postoji
os.makedirs(save_dir, exist_ok=True)

# Sada možete sačuvati model
model_save_path = os.path.join(save_dir, "bert_custom_ner_model.pth")
torch.save(model.state_dict(), model_save_path)


In [13]:
print(save_dir)

C:/Users/nikol/Desktop/MAS/SIR 2/bert/


In [15]:
model.eval()

BertForTokenClassification(
  (bert): BertModel(
    (embeddings): BertEmbeddings(
      (word_embeddings): Embedding(30522, 768, padding_idx=0)
      (position_embeddings): Embedding(512, 768)
      (token_type_embeddings): Embedding(2, 768)
      (LayerNorm): LayerNorm((768,), eps=1e-12, elementwise_affine=True)
      (dropout): Dropout(p=0.1, inplace=False)
    )
    (encoder): BertEncoder(
      (layer): ModuleList(
        (0-11): 12 x BertLayer(
          (attention): BertAttention(
            (self): BertSelfAttention(
              (query): Linear(in_features=768, out_features=768, bias=True)
              (key): Linear(in_features=768, out_features=768, bias=True)
              (value): Linear(in_features=768, out_features=768, bias=True)
              (dropout): Dropout(p=0.1, inplace=False)
            )
            (output): BertSelfOutput(
              (dense): Linear(in_features=768, out_features=768, bias=True)
              (LayerNorm): LayerNorm((768,), eps=1e-12, el

In [17]:
from sklearn.metrics import accuracy_score
import numpy as np

# Postavite model u režim evaluacije
model.eval()

# Inicijalizujte prazne liste za stvarne i predviđene oznake
true_labels = []
predicted_labels = []

# Iterirajte kroz test dataloader
for batch in test_dataloader:
    input_ids, attention_mask, labels = batch
    with torch.no_grad():
        outputs = model(input_ids, attention_mask=attention_mask)

    # Izlazi modela su u formatu logita, koristite argmax da biste dobili predviđene oznake
    predicted_batch_labels = torch.argmax(outputs.logits, dim=2).cpu().numpy()
    true_batch_labels = labels.cpu().numpy()

    # Dodajte oznake za ovaj batch u liste
    predicted_labels.extend(predicted_batch_labels)
    true_labels.extend(true_batch_labels)

# Pretvorite listu u numpy niz za računanje tačnosti
true_labels = np.concatenate(true_labels, axis=0)
predicted_labels = np.concatenate(predicted_labels, axis=0)

# Izračunajte tačnost
accuracy = accuracy_score(true_labels, predicted_labels)

print(f"Tačnost modela na test skupu podataka: {accuracy * 100:.2f}%")


Tačnost modela na test skupu podataka: 95.39%


In [18]:
from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score, confusion_matrix

# True labels su true_labels, a predicted labels su predicted_labels iz prethodnog odgovora

# Tačnost
accuracy = accuracy_score(true_labels, predicted_labels)

# Preciznost
precision = precision_score(true_labels, predicted_labels, average='weighted')

# Odziv
recall = recall_score(true_labels, predicted_labels, average='weighted')

# F1-Skor
f1 = f1_score(true_labels, predicted_labels, average='weighted')

# Matrica konfuzije
confusion = confusion_matrix(true_labels, predicted_labels)

print(f"Tačnost: {accuracy}")
print(f"Preciznost: {precision}")
print(f"Odziv: {recall}")
print(f"F1-Skor: {f1}")
print("Matrica konfuzije:")
print(confusion)


/usr/local/lib/python3.10/dist-packages/sklearn/metrics/_classification.py:1344: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))


Tačnost: 0.9539354838709677
Preciznost: 0.9415873860120019
Odziv: 0.9539354838709677
F1-Skor: 0.9461468696240154
Matrica konfuzije:
[[     0      0      0 ...      0     58      5]
 [     0      0      0 ...      0      5      1]
 [     0      0      0 ...      0     90      8]
 ...
 [     0      0      0 ...      0     19      3]
 [     0      0      0 ...      0    747    100]
 [     0      0      0 ...      0     40 117222]]


In [ ]:
def plot_confusion_matrix(self, gold, tagger_tags):
  labels = labels_srWaC
	confusion_matrix = skmetrics.confusion_matrix(gold, tagger_tags, labels=labels)

	df_cm = pandas.DataFrame(confusion_matrix, index = [i for i in labels], columns = [i for i in labels])
	df_cm.index.name = 'Actual'
	df_cm.columns.name = 'Predicted'
	fig, ax = plt.subplots(figsize=(len(labels), len(labels)))
	seaborn.heatmap(df_cm, annot=True, fmt='', ax=ax)
	plt.savefig(self.model_name + ".png")
	plt.show()

In [19]:
import os

# Putanja do direktorijuma u kojem želite da sačuvate model
save_dir = r"/content/"

# Kreirajte direktorijum ako ne postoji
os.makedirs(save_dir, exist_ok=True)

# Sada možete sačuvati model
model_save_path = os.path.join(save_dir, "bert_custom_ner_model.pth")
torch.save(model.state_dict(), model_save_path)

In [28]:
#torch.save(model, '/content/bertsrWacfinetuned')
sm = torch.load('/content/bertsrWacfinetuned')
sm.eval()

BertForTokenClassification(
  (bert): BertModel(
    (embeddings): BertEmbeddings(
      (word_embeddings): Embedding(30522, 768, padding_idx=0)
      (position_embeddings): Embedding(512, 768)
      (token_type_embeddings): Embedding(2, 768)
      (LayerNorm): LayerNorm((768,), eps=1e-12, elementwise_affine=True)
      (dropout): Dropout(p=0.1, inplace=False)
    )
    (encoder): BertEncoder(
      (layer): ModuleList(
        (0-11): 12 x BertLayer(
          (attention): BertAttention(
            (self): BertSelfAttention(
              (query): Linear(in_features=768, out_features=768, bias=True)
              (key): Linear(in_features=768, out_features=768, bias=True)
              (value): Linear(in_features=768, out_features=768, bias=True)
              (dropout): Dropout(p=0.1, inplace=False)
            )
            (output): BertSelfOutput(
              (dense): Linear(in_features=768, out_features=768, bias=True)
              (LayerNorm): LayerNorm((768,), eps=1e-12, el

In [22]:
model2 = BertForTokenClassification.from_pretrained("bert-base-uncased", num_labels=len(labels_srWaC))
model2.load_state_dict(torch.load("/content/bert_custom_ner_model.pth"))

Some weights of BertForTokenClassification were not initialized from the model checkpoint at bert-base-uncased and are newly initialized: ['classifier.bias', 'classifier.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


<All keys matched successfully>

In [23]:
from transformers import TFBertForSequenceClassification

model_name = 'fine_tuned_bert'

model2.save_pretrained('fine_tuned_bert')
model2 = TFBertForSequenceClassification.from_pretrained(model_name)

OSError: ignored